In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
brandlist = pd.read_csv('../analysis/wordnet_db/wordnet_food_beverages_list.csv', header=None, names=['word'])

In [3]:
reviews = pd.read_json('restaurant_reviews_1900k.json', lines=True)

In [4]:
reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
1,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw
2,3fw2X5bZYeW9xCz_zGhOHg,5,2016-05-07 01:21:02,4,G7XHMxG0bx9oBJNECG4IFg,3,Tracy dessert had a big name in Hong Kong and ...,5,jlu4CztcSxrKx56ba1a5AQ
3,zvO-PJCpNk4fgAVUnExYAA,1,2010-10-05 19:12:35,1,8e9HxxLjjqc9ez5ezzN7iQ,1,This place has gone down hill. Clearly they h...,3,d6xvYpyzcfbF_AZ8vMB7QA
4,8mIrX_LrOnAqWsB5JrOojQ,0,2011-11-30 02:11:15,0,kbtscdyz6lvrtGjD1quQTg,4,"Like walking back in time, every Saturday morn...",0,FIk4lQQu1eTe2EpzQ4xhBA


In [5]:
# reviews.text = reviews.text.str.lower()
brandlist.word = brandlist.word.str.lower()

In [11]:
sample = reviews[:10000]

In [12]:
# Old implementation (regex) - 6x slower
"""
%%time
train_data = []
for r, i in sample.iterrows():
    brands = []
    for brand in brandlist.word:
        for m in re.finditer(r'\b' + brand + r'\b', i.text.lower()):
            brands.append((m.start(0), m.end(0), "PRODUCT"))
    train_data.append((i.review_id, i.text, brands))
"""

'\n%%time\ntrain_data = []\nfor r, i in sample.iterrows():\n    brands = []\n    for brand in brandlist.word:\n        for m in re.finditer(r\'\x08\' + brand + r\'\x08\', i.text.lower()):\n            brands.append((m.start(0), m.end(0), "PRODUCT"))\n    train_data.append((i.review_id, i.text, brands))\n'

In [13]:
%%time
train_data = []
for r, i in sample.iterrows():
    brands = []
    for brand in brandlist.word:
        text = i.text.lower()
        start_index = 0
        while start_index < len(text):
            start_index = text.find(brand, start_index)
            end_index = start_index + len(brand)
            if start_index == -1:
                break
            if not text[start_index-1].isalpha() and (end_index == len(text) or not text[end_index].isalpha()):
                if brand not in ['place', 'restaurant', 'cafe', 'establishment', 'diner']:
                    brands.append((start_index, end_index, "PRODUCT"))
                else:
                    brands.append((start_index, end_index, "ESTABLISHMENT"))
            start_index += len(brand)
    train_data.append((i.review_id, i.text, brands))

CPU times: user 7min 36s, sys: 2.14 s, total: 7min 38s
Wall time: 7min 41s


In [14]:
result = pd.DataFrame(train_data, columns=['review_id', 'text', 'entities'])

In [15]:
result.to_csv('spacy_train_10k.csv')